In [0]:
# ==============================================================================
# CONFIGURAÇÕES E IMPORTS
# ==============================================================================
import requests
import pandas as pd
from datetime import datetime
from pyspark.sql import functions as F
import json

# Configurar catálogo e schema (padrão Unity Catalog)
spark.sql("USE CATALOG previsao_brasileirao")
spark.sql("USE SCHEMA bronze")

# Configurações de logging
print(f"=" * 80)
print(f"INICIANDO INGESTÃO DA CAMADA BRONZE")
print(f"Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Catálogo: {spark.catalog.currentCatalog()}")
print(f"Schema: {spark.catalog.currentDatabase()}")
print(f"=" * 80)

In [0]:
# ==============================================================================
# FUNÇÕES AUXILIARES PARA CHAMADAS À API
# ==============================================================================

def fazer_requisicao_api(url, timeout=30, retries=3):
    """
    Função robusta para fazer requisições HTTP com retry logic
    """
    for tentativa in range(retries):
        try:
            print(f"  Tentativa {tentativa + 1}/{retries} para {url}")
            response = requests.get(url, timeout=timeout)
            response.raise_for_status() # Lança um erro para status codes HTTP 4xx/5xx
            return response.json()
        except requests.exceptions.Timeout:
            print(f"  ⚠️  Timeout na tentativa {tentativa + 1}")
        except requests.exceptions.RequestException as e:
            print(f"  ⚠️  Erro na tentativa {tentativa + 1}: {e}")
        
        if tentativa < retries - 1:
            import time
            time.sleep(2 ** tentativa) # Exponential backoff
    
    raise Exception(f"Falha após {retries} tentativas para {url}")

def adicionar_metadata_ingestao(df):
    """
    Adiciona colunas de metadados de ingestão a um DataFrame Spark
    """
    return df.withColumn("data_ingestao", F.current_timestamp()) \
             .withColumn("fonte_dados", F.lit("API_Cartola_FC"))

print("✅ Funções auxiliares carregadas com sucesso")

In [0]:
# ==============================================================================
# INGESTÃO 1: DADOS DAS RODADAS E PARTIDAS (VERSÃO FINAL E OTIMIZADA)
# ==============================================================================
print("\n[1/4] Iniciando ingestão de dados de TODAS as rodadas disponíveis...")

NOME_TABELA_PARTIDAS = "previsao_brasileirao.bronze.partidas_raw"
TOTAL_RODADAS_CAMPEONATO = 38
RODADAS_PARA_BUSCAR = TOTAL_RODADAS_CAMPEONATO + 5 

try:
    todas_partidas = []
    # --- AJUSTE: Contador para parada inteligente ---
    rodadas_vazias_consecutivas = 0

    for rodada in range(1, RODADAS_PARA_BUSCAR + 1):
        try:
            url_rodada = f"https://api.cartolafc.globo.com/partidas/{rodada}"
            dados_rodada = fazer_requisicao_api(url_rodada)
            
            if 'partidas' in dados_rodada and dados_rodada['partidas']:
                for partida in dados_rodada['partidas']:
                    partida['rodada'] = rodada
                
                todas_partidas.extend(dados_rodada['partidas'])
                print(f"  ✅ Rodada {rodada}: {len(dados_rodada['partidas'])} partidas encontradas.")
                # Se encontramos dados, zeramos o contador de rodadas vazias
                rodadas_vazias_consecutivas = 0
            else:
                print(f"  🟡 Rodada {rodada}: Nenhuma partida encontrada.")
                rodadas_vazias_consecutivas += 1

        except Exception as e:
            print(f"  ⚠️  Rodada {rodada} não disponível ou com erro: {e}")
            rodadas_vazias_consecutivas += 1
        
        # --- AJUSTE: Condição de parada ---
        # Se não encontrarmos dados por 3 rodadas seguidas, paramos o loop.
        if rodadas_vazias_consecutivas >= 3:
            print("\n  ℹ️  Nenhuma partida encontrada por 3 rodadas consecutivas. Encerrando busca antecipadamente.")
            break
            
    if todas_partidas:
        df_partidas_spark = spark.createDataFrame(todas_partidas)
        
        df_partidas_spark.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(NOME_TABELA_PARTIDAS)
        
        print(f"\n✅ SUCESSO! {df_partidas_spark.count()} registros de partidas salvos em '{NOME_TABELA_PARTIDAS}'")
    else:
        print("❌ ERRO CRÍTICO: Nenhuma partida foi encontrada em nenhuma rodada.")
        
except Exception as e:
    print(f"❌ ERRO GERAL na ingestão de partidas: {e}")
    raise

In [0]:
# ==============================================================================
# INGESTÃO 2: STATUS E ESTATÍSTICAS DOS JOGADORES
# ==============================================================================
print("\n[2/4] Iniciando ingestão de status dos jogadores...")

# --- AJUSTE APLICADO AQUI: DEFINIÇÃO EXPLÍCITA DO SCHEMA ---
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType

# Definimos a estrutura exata dos dados, tratando campos numéricos que podem
# ter decimais como DoubleType para evitar conflitos.
schema_jogadores = StructType([
    StructField("atleta_id", LongType(), True),
    StructField("apelido", StringType(), True),
    StructField("clube_id", LongType(), True),
    StructField("foto", StringType(), True),
    StructField("jogos_num", LongType(), True),
    StructField("media_num", DoubleType(), True),
    StructField("minimo_para_valorizar", DoubleType(), True),
    StructField("nome", StringType(), True),
    StructField("pontos_num", DoubleType(), True),
    StructField("posicao_id", LongType(), True),
    StructField("preco_num", DoubleType(), True),
    StructField("rodada_id", LongType(), True),
    StructField("scout", StringType(), True), # O scout vem como um objeto, mas o salvamos como JSON string
    StructField("slug", StringType(), True),
    StructField("status_id", LongType(), True),
    StructField("variacao_num", DoubleType(), True),
    StructField("gato_mestre", StructType([ # Campo aninhado
        StructField("epoca_maior_pontuador", LongType(), True),
        StructField("gato_mestre", StringType(), True),
        StructField("media_pontos_mandante", DoubleType(), True),
        StructField("media_pontos_visitante", DoubleType(), True),
        StructField("media_pontos_geral", DoubleType(), True)
    ]), True)
])
# --- FIM DO AJUSTE ---

URL_MERCADO = "https://api.cartolafc.globo.com/atletas/mercado"
NOME_TABELA_JOGADORES = "previsao_brasileirao.bronze.jogadores_status_raw"

try:
    dados_mercado = fazer_requisicao_api(URL_MERCADO)
    
    rodada_atual_jogadores = dados_mercado.get('rodada_atual')
    atletas = dados_mercado.get('atletas', [])
    for atleta in atletas:
        atleta['rodada_id'] = rodada_atual_jogadores
        # Garantir que o scout seja uma string JSON para compatibilidade com o schema
        if 'scout' in atleta and atleta['scout'] is not None:
            atleta['scout'] = json.dumps(atleta['scout'])
        else:
            atleta['scout'] = None

    if atletas:
        # Usando o schema explícito ao criar o DataFrame
        df_jogadores_spark = spark.createDataFrame(atletas, schema=schema_jogadores)
        
        df_jogadores_spark = adicionar_metadata_ingestao(df_jogadores_spark)
        
        df_jogadores_spark.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(NOME_TABELA_JOGADORES)
        
        print(f"\n✅ SUCESSO! {len(atletas)} jogadores salvos em '{NOME_TABELA_JOGADORES}'")
        display(df_jogadores_spark.limit(5))
    else:
        print("⚠️ Nenhum jogador encontrado na API do mercado.")

except Exception as e:
    print(f"❌ ERRO na ingestão de jogadores: {e}")
    raise

In [0]:
# ==============================================================================
# INGESTÃO 3: INFORMAÇÕES DOS CLUBES
# ==============================================================================
print("\n[3/4] Iniciando ingestão de informações dos clubes...")

# --- AJUSTE APLICADO AQUI: DEFINIÇÃO EXPLÍCITA DO SCHEMA ---
from pyspark.sql.types import StructType, StructField, LongType, StringType, MapType

# Definimos a estrutura dos dados dos clubes.
# O campo 'escudos' é um objeto JSON (key-value), então usamos MapType.
schema_clubes = StructType([
    StructField("id", LongType(), True),
    StructField("nome", StringType(), True),
    StructField("abreviacao", StringType(), True),
    StructField("slug", StringType(), True),
    StructField("escudos", MapType(StringType(), StringType()), True),
    StructField("nome_fantasia", StringType(), True)
])
# --- FIM DO AJUSTE ---

URL_CLUBES = "https://api.cartolafc.globo.com/clubes"
NOME_TABELA_CLUBES = "previsao_brasileirao.bronze.clubes_info_raw"

try:
    dados_clubes = fazer_requisicao_api(URL_CLUBES)
    
    # A API retorna um dicionário, convertemos para uma lista de objetos
    lista_clubes = [clube for clube_id, clube in dados_clubes.items()]
    
    if lista_clubes:
        # --- AJUSTE APLICADO AQUI ---
        # Usando createDataFrame com o schema explícito, que é compatível com Serverless
        df_clubes_spark = spark.createDataFrame(lista_clubes, schema=schema_clubes)
        # --- FIM DO AJUSTE ---
        
        df_clubes_spark = adicionar_metadata_ingestao(df_clubes_spark)
        
        df_clubes_spark.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(NOME_TABELA_CLUBES)
        
        print(f"\n✅ SUCESSO! {len(lista_clubes)} clubes salvos em '{NOME_TABELA_CLUBES}'")
        display(df_clubes_spark)
        
except Exception as e:
    print(f"❌ ERRO na ingestão de clubes: {e}")
    raise

In [0]:
# ==============================================================================
# INGESTÃO 4: PONTUAÇÕES DOS ATLETAS POR RODADA (HISTÓRICO)
# ==============================================================================
print("\n[4/4] Iniciando ingestão de pontuações históricas...")

# --- AJUSTE APLICADO: DEFINIÇÃO EXPLÍCITA DO SCHEMA ---
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, MapType

# Definimos a estrutura dos dados de pontuação para evitar erros de tipo
schema_pontuacoes = StructType([
    StructField("rodada_id", LongType(), True),
    StructField("atleta_id", LongType(), True),
    StructField("apelido", StringType(), True),
    StructField("clube_id", LongType(), True),
    StructField("posicao_id", LongType(), True),
    StructField("pontos_num", DoubleType(), True),
    StructField("preco_num", DoubleType(), True),
    StructField("variacao_num", DoubleType(), True),
    StructField("scout", MapType(StringType(), LongType()), True)
])
# --- FIM DO AJUSTE ---


URL_MERCADO = "https://api.cartolafc.globo.com/atletas/mercado"
NOME_TABELA_PONTUACOES = "previsao_brasileirao.bronze.pontuacoes_historico_raw"

try:
    # --- AJUSTE APLICADO: LIDANDO COM MERCADO FECHADO ---
    # Quando o mercado do Cartola está fechado, a API não retorna a 'rodada_atual'.
    # Para fins de desenvolvimento, vamos definir um valor fixo.
    # Em um ambiente de produção, você poderia verificar o status do mercado antes de prosseguir.
    
    # Linha original (descomente para usar em produção quando o mercado estiver aberto)
    # dados_mercado_pontos = fazer_requisicao_api(URL_MERCADO)
    # rodada_atual = dados_mercado_pontos.get('rodada_atual')
    
    # Linha para desenvolvimento (define a rodada 10 como a "atual" para buscar o histórico)
    rodada_atual = 10 
    
    if not rodada_atual:
        raise ValueError("Não foi possível obter a rodada atual da API do mercado.")
    print(f"  ℹ️  Rodada atual para busca de histórico (definida manualmente): {rodada_atual}")
    # --- FIM DO AJUSTE ---

    todas_pontuacoes = []
    # Busca das últimas 10 rodadas (ou desde a 1) até a rodada atual
    for rodada in range(max(1, rodada_atual - 10), rodada_atual + 1):
        try:
            url_pontos = f"https://api.cartolafc.globo.com/atletas/pontuados/{rodada}"
            dados_pontos = fazer_requisicao_api(url_pontos)
            
            if 'atletas' in dados_pontos:
                for atleta_id, atleta_dados in dados_pontos['atletas'].items():
                    atleta_dados['rodada_id'] = rodada
                    atleta_dados['atleta_id'] = int(atleta_id)
                    todas_pontuacoes.append(atleta_dados)
                print(f"  ✅ Rodada {rodada}: {len(dados_pontos['atletas'])} pontuações")
        except Exception as e:
            print(f"  ⚠️  Dados da rodada {rodada} não disponíveis ou com erro: {e}")
            continue
    
    if todas_pontuacoes:
        # --- AJUSTE APLICADO: Usando createDataFrame (compatível com Serverless) ---
        df_pontos_spark = spark.createDataFrame(todas_pontuacoes, schema=schema_pontuacoes)
        
        df_pontos_spark = adicionar_metadata_ingestao(df_pontos_spark)
        
        df_pontos_spark.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(NOME_TABELA_PONTUACOES)
        
        print(f"\n✅ SUCESSO! {len(todas_pontuacoes)} registros de pontuações salvos em '{NOME_TABELA_PONTUACOES}'")
        display(df_pontos_spark.limit(5))
    else:
        print("⚠️  Nenhuma pontuação histórica encontrada")
        
except Exception as e:
    print(f"❌ ERRO na ingestão de pontuações: {e}")
    raise

In [0]:
# ==============================================================================
# VALIDAÇÃO E SUMÁRIO DA INGESTÃO
# ==============================================================================
print("\n" + "=" * 80)
print("SUMÁRIO DA INGESTÃO - CAMADA BRONZE")
print("=" * 80)

tabelas_bronze = spark.sql(f"SHOW TABLES IN previsao_brasileirao.bronze").collect()

if not tabelas_bronze:
    print("Nenhuma tabela encontrada no schema 'previsao_brasileirao.bronze'.")
else:
    for tabela in tabelas_bronze:
        nome_tabela_completo = f"previsao_brasileirao.bronze.{tabela['tableName']}"
        try:
            df_validacao = spark.table(nome_tabela_completo)
            count = df_validacao.count()
            
            print(f"\n📊 Tabela: {nome_tabela_completo}")
            print(f"   Registros: {count:,}")
            print(f"   Colunas: {len(df_validacao.columns)}")
        except Exception as e:
            print(f"\n❌ Erro ao validar a tabela {nome_tabela_completo}: {e}")

print("\n" + "=" * 80)
print("✅ VALIDAÇÃO DA CAMADA BRONZE CONCLUÍDA!")
print(f"Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

In [0]:
# CÉLULA DE DIAGNÓSTICO
import requests
import json

try:
    # Este é o endpoint mais confiável para saber o status real do mercado
    status_url = "https://api.cartolafc.globo.com/mercado/status"
    response = requests.get(status_url)
    response.raise_for_status()
    dados_status = response.json()

    print("--- Diagnóstico da API do Cartola FC ---")
    print(f"Status do Mercado: {dados_status.get('status_mercado')} (1=Aberto, 2=Fechado, 3=Em atualização, 4=Processando)")
    print(f"Rodada Atual segundo a API: {dados_status.get('rodada_atual')}")
    print(f"Fechamento do Mercado: {dados_status.get('fechamento', {}).get('dia')}/{dados_status.get('fechamento', {}).get('mes')} às {dados_status.get('fechamento', {}).get('hora')}:{dados_status.get('fechamento', {}).get('minuto')}")

except Exception as e:
    print(f"Erro ao consultar a API de status: {e}")

In [0]:
# CÉLULA DE DIAGNÓSTICO FINAL
import requests

print("--- Verificando o conteúdo real das rodadas na API ---")

# Rodada ATUAL (deve funcionar)
url_rodada_29 = "https://api.cartolafc.globo.com/partidas/29"
try:
    dados_29 = requests.get(url_rodada_29).json()
    num_partidas_29 = len(dados_29.get('partidas', []))
    print(f"✅ Rodada 29: Encontradas {num_partidas_29} partidas.")
except Exception as e:
    print(f"❌ Erro ao buscar Rodada 29: {e}")

# Rodada FUTURA (provavelmente vai falhar ou vir vazia)
url_rodada_30 = "https://api.cartolafc.globo.com/partidas/30"
try:
    dados_30 = requests.get(url_rodada_30).json()
    num_partidas_30 = len(dados_30.get('partidas', []))
    print(f"✅ Rodada 30: Encontradas {num_partidas_30} partidas.")
except Exception as e:
    print(f"❌ Erro ao buscar Rodada 30: {e}")